## What is Tensorflow?

TensorFlow is an open source software library for numerical computation using data flow graphs

TensorFlow programs are usually structured into a construction phase, that assembles a graph, and an execution phase that uses a session to execute ops in the graph

A TF program often has 2 phases:

- Assemble a graph
- Use a session to execute operations in the graph

## Type of Tensor in Tensorflow

The main types of tensors are:

- tf.Variable / tf.get_variable

- tf.constant

- tf.placeholder


## Basic Code Structure - Graphs

- Constants are fixed value tensors - not trainable

- Variables are tensors initialized in a session - trainable

- Placeholders are tensors of values that are unknown during the graph
construction, but passed as input during a session

- Ops are functions on tensors
 

## Activity: first construct a multiplication and then execute it in Tensorflow

In [3]:
import tensorflow as tf

input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.multiply(input1, input2)

with tf.Session() as sess:
    print(sess.run(output, feed_dict={input1: [7.], input2: [2.]}))

[14.]


## Visualization of Tensors and ops

<img src="tensorflow_graph_tensor_ops.png" width="500" height="500">

## Activity: Write a linear regression with Tensorflow

Change the indentation of `print(sess.run(W))` and see the result

In [7]:
from __future__ import print_function

import tensorflow as tf
import numpy
import matplotlib.pyplot as plt
rng = numpy.random

# Parameters
learning_rate = 0.01
training_epochs = 1000
display_step = 50

# Training Data
train_X = numpy.asarray([3.3,4.4,5.5,6.71,6.93,4.168,9.779,6.182,7.59,2.167,
                         7.042,10.791,5.313,7.997,5.654,9.27,3.1])
train_Y = numpy.asarray([1.7,2.76,2.09,3.19,1.694,1.573,3.366,2.596,2.53,1.221,
                         2.827,3.465,1.65,2.904,2.42,2.94,1.3])
n_samples = train_X.shape[0]

# tf Graph Input
X = tf.placeholder("float")
Y = tf.placeholder("float")

# Set model weights
W = tf.Variable(rng.randn(), name="weight")
b = tf.Variable(rng.randn(), name="bias")

# Construct a linear model
pred = tf.add(tf.multiply(X, W), b)

# Mean squared error
cost = tf.reduce_sum(tf.pow(pred-Y, 2))/(2*n_samples)
# Gradient descent
#  Note, minimize() knows to modify W and b because Variable objects are trainable=True by default
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # Fit all training data
    for epoch in range(training_epochs):
        for (x, y) in zip(train_X, train_Y):
            sess.run(optimizer, feed_dict={X: x, Y: y})
    print(sess.run(W))

0.32856944


## Activity: Build a NN for Stock Market Prediction with Tensorflow

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Import data
data = pd.read_csv('data_stocks.csv')

# Drop date variable
data = data.drop(['DATE'], 1)

# Dimensions of dataset
n = data.shape[0]
p = data.shape[1]

# Make data a np.array
data = data.values

# Training and test data
train_start = 0
train_end = int(np.floor(0.8*n))
test_start = train_end + 1
test_end = n
data_train = data[np.arange(train_start, train_end), :]
data_test = data[np.arange(test_start, test_end), :]

# Scale data
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(data_train)
data_train = scaler.transform(data_train)
data_test = scaler.transform(data_test)

# Build X and y
X_train = data_train[:, 1:]
y_train = data_train[:, 0]
X_test = data_test[:, 1:]
y_test = data_test[:, 0]

# Number of stocks in training data
n_stocks = X_train.shape[1]

# Neurons
n_neurons_1 = 1024
n_neurons_2 = 512
n_neurons_3 = 256
n_neurons_4 = 128

# Session
net = tf.InteractiveSession()

# Placeholder
X = tf.placeholder(dtype=tf.float32, shape=[None, n_stocks])
Y = tf.placeholder(dtype=tf.float32, shape=[None])

# Initializers
sigma = 1
weight_initializer = tf.variance_scaling_initializer(mode="fan_avg", distribution="uniform", scale=sigma)
bias_initializer = tf.zeros_initializer()

# Hidden weights
W_hidden_1 = tf.Variable(weight_initializer([n_stocks, n_neurons_1]))
bias_hidden_1 = tf.Variable(bias_initializer([n_neurons_1]))
W_hidden_2 = tf.Variable(weight_initializer([n_neurons_1, n_neurons_2]))
bias_hidden_2 = tf.Variable(bias_initializer([n_neurons_2]))
W_hidden_3 = tf.Variable(weight_initializer([n_neurons_2, n_neurons_3]))
bias_hidden_3 = tf.Variable(bias_initializer([n_neurons_3]))
W_hidden_4 = tf.Variable(weight_initializer([n_neurons_3, n_neurons_4]))
bias_hidden_4 = tf.Variable(bias_initializer([n_neurons_4]))

# Output weights
W_out = tf.Variable(weight_initializer([n_neurons_4, 1]))
bias_out = tf.Variable(bias_initializer([1]))

# Hidden layer
hidden_1 = tf.nn.relu(tf.add(tf.matmul(X, W_hidden_1), bias_hidden_1))
hidden_2 = tf.nn.relu(tf.add(tf.matmul(hidden_1, W_hidden_2), bias_hidden_2))
hidden_3 = tf.nn.relu(tf.add(tf.matmul(hidden_2, W_hidden_3), bias_hidden_3))
hidden_4 = tf.nn.relu(tf.add(tf.matmul(hidden_3, W_hidden_4), bias_hidden_4))

# Output layer (transpose!)
out = tf.transpose(tf.add(tf.matmul(hidden_4, W_out), bias_out))

# Cost function
mse = tf.reduce_mean(tf.squared_difference(out, Y))

# Optimizer
opt = tf.train.AdamOptimizer().minimize(mse)

# Init
net.run(tf.global_variables_initializer())

# Setup plot
plt.ion()
fig = plt.figure()
ax1 = fig.add_subplot(111)
line1, = ax1.plot(y_test)
line2, = ax1.plot(y_test * 0.5)
plt.show()

# Fit neural net
batch_size = 256
mse_train = []
mse_test = []

# Run
epochs = 10
for e in range(epochs):

    # Shuffle training data
    shuffle_indices = np.random.permutation(np.arange(len(y_train)))
    X_train = X_train[shuffle_indices]
    y_train = y_train[shuffle_indices]

    # Minibatch training
    for i in range(0, len(y_train) // batch_size):
        start = i * batch_size
        batch_x = X_train[start:start + batch_size]
        batch_y = y_train[start:start + batch_size]
        # Run optimizer with batch
        net.run(opt, feed_dict={X: batch_x, Y: batch_y})

        # Show progress
        if np.mod(i, 50) == 0:
            # MSE train and test
            mse_train.append(net.run(mse, feed_dict={X: X_train, Y: y_train}))
            mse_test.append(net.run(mse, feed_dict={X: X_test, Y: y_test}))
            print('MSE Train: ', mse_train[-1])
            print('MSE Test: ', mse_test[-1])
            # Prediction
            pred = net.run(out, feed_dict={X: X_test})
            line2.set_ydata(pred)
            plt.title('Epoch ' + str(e) + ', Batch ' + str(i))
            plt.pause(0.01)

## While loop in Tensorflow

In [11]:
import tensorflow as tf

i = tf.constant(0)
c = lambda i: tf.less(i, 10)
b = lambda i: tf.add(i, 1)
r = tf.while_loop(c, b, [i])
with tf.Session() as sess:
    print(sess.run(r))

10


In [9]:
import tensorflow as tf

a, b = tf.while_loop(lambda x, y: x < 30, lambda x, y: (x * 3, y * 2), [2, 3])

# Run the while loop and get the resulting values.
with tf.Session() as sess:
    print(sess.run([a, b]))

[54, 24]


## Subgraph in Tensorflow

<img src="tensorflow_subgraph.png" width="500" height="500">

In [ ]:
with tf.device("/gpu:0"):
    # Setup operations

with tf.Session() as sess:
    # Run your code

## Without specifing which operations on which hardware

In [ ]:
import tensorflow as tf

c = []
a = tf.get_variable("a", [2, 2], initializer=tf.random_uniform_initializer(-1, 1))
b = tf.get_variable("b", [2, 2], initializer=tf.random_uniform_initializer(-1, 1))

c.append(tf.matmul(a, b))

c.append(a + b)

d = tf.add_n(c)


init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(c))
    print(sess.run(d))

## specifing which operations on which hardware

In [ ]:
# https://jhui.github.io/2017/03/07/TensorFlow-GPU/

import tensorflow as tf

c = []
a = tf.get_variable(f"a", [2, 2], initializer=tf.random_uniform_initializer(-1, 1))
b = tf.get_variable(f"b", [2, 2], initializer=tf.random_uniform_initializer(-1, 1))

with tf.device('/gpu:0'):
    c.append(tf.matmul(a, b))

with tf.device('/gpu:1'):
    c.append(a + b)

with tf.device('/cpu:0'):
    d = tf.add_n(c)

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True, allow_soft_placement=True))

init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(d))


## Tensor manipulation in TF

<img src="tensorflow_scatter_gather.png" width="500" height="500">

In [3]:
import tensorflow as tf

a = tf.Variable(initial_value=tf.constant([1, 2, 3, 4, 5, 6, 7, 8]))
indices = tf.constant([[4], [3], [1], [7]])
updates = tf.constant([9, 10, 11, 12])
update = tf.scatter_nd_update(a, indices, updates)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(update))

[ 1 11  3 10  9  6  7 12]


In [11]:
import tensorflow as tf
# Zeros matrix
num = tf.get_variable('num111111', shape=[5, 3], initializer=tf.zeros_initializer(), dtype=tf.float32)
updates = tf.ones([2, 3], dtype=tf.float32)
num = tf.scatter_nd_update(num, [[0], [4]], updates)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(num))

[[1. 1. 1.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [1. 1. 1.]]


In [2]:
import tensorflow as tf
# Zeros matrix
num = tf.get_variable('num1', shape=[5, 3], initializer=tf.zeros_initializer(), dtype=tf.float32)
# Looping variable
i = tf.constant(0, dtype=tf.int32)


def body(i, num, j):
    # Update values
    updates = tf.ones([1, 3], dtype=tf.float32)
    for i in range(sess.run(tf.add(i, j))):
        num = tf.scatter_nd_update(num, [[i]], updates)
    return num


sess = tf.Session()
sess.run(tf.global_variables_initializer())
print(sess.run(body(i, num, 2)))

[[1. 1. 1.]
 [1. 1. 1.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [12]:
import tensorflow as tf
# Zeros matrix
num = tf.get_variable('numb', shape=[5, 3], initializer=tf.zeros_initializer(), dtype=tf.float32)
# Looping variable
#i = tf.constant(0, dtype=tf.int32)


def body(num, j):
    # Update values
    updates = tf.ones([1, 3], dtype=tf.float32)
    for i in range(j):
        num = tf.scatter_nd_update(num, [[i]], updates)
    return num


sess = tf.Session()
sess.run(tf.global_variables_initializer())
print(sess.run(body(num, 2)))

[[1. 1. 1.]
 [1. 1. 1.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


## Resources:

- http://www.cs.tau.ac.il/~joberant/teaching/advanced_nlp_spring_2018/files/tensorflow_tutorial.pdf
